# Utilizzo della libreria Backtester

Questo notebook mostra come utilizzare la libreria di backtesting che abbiamo creato per testare strategie di trading con le Bollinger Bands.

## 1. Importazione delle librerie

In [ ]:
# Importa i moduli della libreria backtester
import sys
sys.path.append('/home/edocame/Desktop/bollingerBands/notebooks')  # Aggiungi il path

from backtester import data_loader, indicators, backtest_engine, visualization, utils
import pandas as pd
import numpy as np

# Importa anche matplotlib per i grafici
import matplotlib.pyplot as plt
%matplotlib inline

## 2. Caricamento e preparazione dei dati

In [ ]:
# Carica i dati tick dal file Parquet
file_path = '/home/edocame/Desktop/data_python/03_BID_ASK_DATA/EURCHF_oanda_alltime.parquet'
tick_data = data_loader.load_parquet_data(file_path)

print(f"Dati caricati: {len(tick_data)} righe")
print(f"Periodo: {tick_data.index.min()} - {tick_data.index.max()}")
print(f"Colonne: {tick_data.columns.tolist()}")
print("\nPrime righe:")
print(tick_data.head())

In [ ]:
# Assicurati che la colonna 'datetime' sia di tipo datetime e impostala come indice
if not pd.api.types.is_datetime64_any_dtype(tick_data['datetime']):
    tick_data['datetime'] = pd.to_datetime(tick_data['datetime'], errors='coerce')
    print('Colonna datetime convertita in datetime.')
else:
    print('Colonna datetime già in formato datetime.')

# Imposta la colonna 'datetime' come indice
if tick_data.index.name != 'datetime':
    tick_data = tick_data.set_index('datetime')
    print('Colonna datetime impostata come indice.')
else:
    print('Colonna datetime già impostata come indice.')

# Mostra le prime righe per conferma
print(tick_data.head())

In [ ]:
# Prepara i dati a minuto
minute_data = data_loader.prepare_minute_data(tick_data, resample_rule='1T')

print(f"Dati a minuto: {len(minute_data)} righe")
print(f"Periodo: {minute_data.index.min()} - {minute_data.index.max()}")
print("\nPrime righe:")
print(minute_data.head())

## 3. Calcolo degli indicatori tecnici

In [ ]:
# Calcola le Bollinger Bands
window = 1440  # 24 ore = 1440 minuti
num_std_dev = 1.0

data_with_bands = indicators.bollinger_bands(
    minute_data, 
    price_column='midprice',
    window=window, 
    num_std_dev=num_std_dev
)

# Rimuovi le righe con valori NaN
complete_data = data_with_bands.dropna()

print(f"Dati completi per il backtest: {len(complete_data)} righe")
print(f"Colonne disponibili: {complete_data.columns.tolist()}")
print("\nPrime righe con indicatori:")
print(complete_data.head())

## 4. Esecuzione del backtest

In [ ]:
# Crea e esegui il backtest
backtester = backtest_engine.Backtest(complete_data)
results_data = backtester.run()

# Stampa il riassunto delle performance
backtester.print_performance_summary()

## 5. Analisi dei risultati

In [ ]:
# Ottieni i risultati come DataFrame
trades_df = backtester.get_trades_dataframe()

if not trades_df.empty:
    print(f"Numero di trade: {len(trades_df)}")
    print("\nPrimi 10 trade:")
    print(trades_df.head(10))
    
    # Statistiche aggiuntive
    win_loss_stats = utils.calculate_win_loss_ratio(trades_df['PnL'])
    print("\nStatistiche Win/Loss:")
    for key, value in win_loss_stats.items():
        print(f"  {key}: {value:.2f}")
else:
    print("Nessun trade eseguito.")

## 6. Visualizzazione dei risultati

In [ ]:
# Visualizza i prezzi con le Bollinger Bands
# Prendi un subset dei dati per una visualizzazione più chiara
subset_data = complete_data.iloc[-2000:]  # Ultimi 2000 punti

visualization.plot_price_with_bollinger_bands(
    subset_data, 
    title="EURCHF con Bollinger Bands (ultimi 2000 punti)"
)

In [ ]:
# Visualizza il PnL cumulativo
if not trades_df.empty:
    visualization.plot_cumulative_pnl(
        trades_df, 
        title="PnL Cumulativo - Strategia Bollinger Bands"
    )

In [ ]:
# Visualizza la distribuzione dei trade
if not trades_df.empty:
    visualization.plot_trade_distribution(
        trades_df, 
        title="Distribuzione PnL dei Trade"
    )

## 7. Grafici interattivi (Plotly)

In [ ]:
# Grafico interattivo dei prezzi con Bollinger Bands
visualization.plot_interactive_price_bands(
    subset_data, 
    title="EURCHF con Bollinger Bands - Grafico Interattivo"
)

In [ ]:
# Grafico interattivo del PnL cumulativo
if not trades_df.empty:
    visualization.plot_interactive_cumulative_pnl(
        trades_df, 
        title="PnL Cumulativo - Grafico Interattivo"
    )

## 8. Confronto con dati reali (se disponibili)

In [ ]:
# Carica i dati di balance reali per confronto
try:
    balance_file = '/home/edocame/Desktop/bollingerBands/DATA/eurchf_1440_01.csv'
    balance_data = data_loader.load_balance_data(balance_file)
    
    print(f"Dati balance caricati: {len(balance_data)} righe")
    print(f"Periodo balance: {balance_data.index.min()} - {balance_data.index.max()}")
    
    # Confronta con i risultati del backtest
    if not trades_df.empty:
        # Trova il periodo comune tra backtest e balance
        # Assumiamo che trades_df abbia una colonna 'Exit_Time' con i timestamp
        if 'Exit_Time' in trades_df.columns:
            backtest_start = trades_df['Exit_Time'].min()
            backtest_end = trades_df['Exit_Time'].max()
        else:
            # Se non c'è Exit_Time, usa l'indice del DataFrame completo
            backtest_start = complete_data.index.min()
            backtest_end = complete_data.index.max()
        
        balance_start = balance_data.index.min()
        balance_end = balance_data.index.max()
        
        # Calcola il periodo comune
        common_start = max(backtest_start, balance_start)
        common_end = min(backtest_end, balance_end)
        
        print(f"\nPeriodo backtest: {backtest_start} - {backtest_end}")
        print(f"Periodo balance: {balance_start} - {balance_end}")
        print(f"Periodo comune: {common_start} - {common_end}")
        
        # Verifica che esista un periodo comune
        if common_start <= common_end:
            # Filtra i dati al periodo comune
            if 'Exit_Time' in trades_df.columns:
                trades_common = trades_df[
                    (trades_df['Exit_Time'] >= common_start) & 
                    (trades_df['Exit_Time'] <= common_end)
                ]
            else:
                # Se non c'è Exit_Time, crea un dataset filtrato basato sui timestamp
                trades_common = trades_df.copy()
                
            balance_common = balance_data.loc[common_start:common_end]
            
            print(f"\nTrade nel periodo comune: {len(trades_common)}")
            print(f"Righe balance nel periodo comune: {len(balance_common)}")
            
            # Crea il grafico di confronto solo per il periodo comune
            if len(trades_common) > 0 and len(balance_common) > 0:
                visualization.compare_backtest_vs_real_balance(
                    trades_common, 
                    balance_common, 
                    title=f"Confronto: Backtest vs Balance Reale ({common_start.strftime('%Y-%m-%d')} - {common_end.strftime('%Y-%m-%d')})"
                )
                
                # Mostra anche alcune statistiche di confronto
                print(f"\n=== STATISTICHE PERIODO COMUNE ===")
                print(f"PnL finale backtest: {trades_common['Cumulative_PnL'].iloc[-1]:.2f} pips")
                print(f"Balance finale reale: {balance_common['<BALANCE>'].iloc[-1]:.2f}")
                
                # Calcola la correlazione se possibile
                if len(trades_common) > 1 and len(balance_common) > 1:
                    # Resample per allineare le frequenze
                    try:
                        balance_daily = balance_common['<BALANCE>'].resample('D').last().dropna()
                        trades_daily = trades_common.set_index('Exit_Time')['Cumulative_PnL'].resample('D').last().dropna()
                        
                        # Trova l'intersezione delle date
                        common_dates = balance_daily.index.intersection(trades_daily.index)
                        
                        if len(common_dates) > 2:
                            corr = balance_daily.loc[common_dates].corr(trades_daily.loc[common_dates])
                            print(f"Correlazione giornaliera: {corr:.3f}")
                        else:
                            print("Insufficienti dati per calcolare la correlazione")
                    except Exception as e:
                        print(f"Errore nel calcolo della correlazione: {e}")
            else:
                print("ERRORE: Nessun dato disponibile nel periodo comune filtrato")
        else:
            print("ERRORE: Non ci sono date in comune tra backtest e balance!")
    else:
        print("Nessun trade disponibile per il confronto")
        
except Exception as e:
    print(f"Errore nel caricamento dei dati balance: {e}")

## 9. Dashboard completo

In [ ]:
# Crea un dashboard completo con tutti i grafici
if not trades_df.empty:
    visualization.create_performance_dashboard(
        subset_data, 
        trades_df, 
        backtester.performance_metrics
    )

## 10. Test con parametri diversi

In [ ]:
# Forza il reload del modulo per assicurarsi che le nuove funzioni siano disponibili
import importlib
import sys

# Reload del modulo backtest_engine per caricare le nuove funzioni
if 'backtester.backtest_engine' in sys.modules:
    importlib.reload(sys.modules['backtester.backtest_engine'])

# Importa le funzioni di ottimizzazione dal backtest_engine
from backtester.backtest_engine import optimize_parameters, plot_top_equity_curves

# Esegui l'ottimizzazione dei parametri con multiprocessing
print("Avvio ottimizzazione parametri...")
parameter_results = optimize_parameters(
    minute_data=minute_data,
    window_start=60,
    window_stop=14400,
    window_step=60,
    std_start=0.2,
    std_stop=4.0,
    std_step=0.2,
    price_column='midprice'
)

# Mostra i risultati completi
print("\n=== TUTTI I RISULTATI ===")
print(parameter_results.to_string(index=False))

# Plotta le top 5 equity curves usando la funzione dedicata
print("\nGenerazione grafici delle migliori equity curves...")
plot_top_equity_curves(
    results_df=parameter_results,
    minute_data=minute_data,
    top_n=5,
    price_column='midprice'
)

## Walk forward opt

In [ ]:
# Importa le funzioni di Walk Forward Optimization dal modulo dedicato
from backtester.walk_forward import walk_forward_optimization, plot_wfo_results

# Execute Walk Forward Optimization with proper error handling
try:
    print("Starting Walk Forward Optimization...")
    wfo_results = walk_forward_optimization(
        minute_data=minute_data,
        lookback_days=90,  # Use 90 days of data for optimization
        optimization_interval_days=14,  # Re-optimize every 14 days
        window_start=400,
        window_stop=14400,
        window_step=100,
        std_start=0.5,
        std_stop=3.0,
        std_step=0.5,
        price_column='midprice'
    )
    
    # Plot and analyze results
    plot_wfo_results(wfo_results)
    
except Exception as e:
    print(f"Error during Walk Forward Optimization: {e}")
    print("Please check your data and parameters.")